In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv
/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv
/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv
/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv


**Libraries Required**

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge # Ridge regression model
import tensorflow as tf
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split,KFold
import warnings
warnings.filterwarnings("ignore")

**Datasources**

In [3]:
prev_train_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
target = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [4]:
test=pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

**Custom weights to existing columns**

In [5]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] * 4.0
prev_train_df["threat"] = prev_train_df["threat"] *4.5
prev_train_df["identity_hate"] = prev_train_df["identity_hate"] * 3.5

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic", "obscene", "threat", "insult",
                                              "identity_hate"]].sum(axis = 1).astype(np.int64)

In [6]:
train_df = prev_train_df[["comment_text", "total_toxic"]]

**Data cleaning--1**

In [7]:
train_df["Comment"] = train_df['comment_text'].str.replace('[^\w\s\n]','')
train_df.head()

,comment_text,total_toxic,Comment
0,Explanation\nWhy the edits made under my usern...,0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...,0,Daww He matches this background colour Im seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man Im really not trying to edit war Its j...
3,"""\nMore\nI can't make any real suggestions on ...",0,\nMore\nI cant make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [8]:
train_df.drop(['comment_text'],axis=1)

,total_toxic,Comment
0,0,Explanation\nWhy the edits made under my usern...
1,0,Daww He matches this background colour Im seem...
2,0,Hey man Im really not trying to edit war Its j...
3,0,\nMore\nI cant make any real suggestions on im...
4,0,You sir are my hero Any chance you remember wh...
...,...,...
159566,0,And for the second time of asking when your vi...
159567,0,You should be ashamed of yourself \n\nThat is ...
159568,0,Spitzer \n\nUmm theres no actual article for p...
159569,0,And it looks like it was actually you who put ...


In [9]:
text=train_df.loc[1:70001,['Comment']]
target=train_df.loc[1:70000,['total_toxic']]

In [10]:
train_df['text'] = train_df['Comment'].apply(lambda x:x.lower())

**Data cleaning --2**

In [11]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [12]:
#removing the stop words from the corpus
train_df['filter1'] =train_df['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

In [13]:
train_df['filter1']

0         explanation edits made username hardcore metal...
1         daww matches background colour im seemingly st...
2         hey man im really trying edit war guy constant...
3         cant make real suggestions improvement wondere...
4                       sir hero chance remember page thats
                                ...                        
159566    second time asking view completely contradicts...
159567       ashamed horrible thing put talk page 128611993
159568    spitzer umm theres actual article prostitution...
159569    looks like actually put speedy first version d...
159570    really dont think understand came idea bad rig...
Name: filter1, Length: 159571, dtype: object

**Vectorization**

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["filter1"])

**Data selection**

In [15]:
X=tfv[1:70001]
Y=train_df.loc[1:70000,['total_toxic']]

**Splitting data into train and test**

In [16]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state = 0)

**Ridge modelling**

In [17]:
rr_model = Ridge(alpha=0.5)
rr_model.fit(x_train,y_train)

Ridge(alpha=0.5)

In [18]:
tfv_comments = tfvec.transform(test["text"])
pred = rr_model.predict(tfv_comments)

In [19]:
sub = pd.DataFrame()
sub["comment_id"] = test["comment_id"]
sub["score"] = pred*3.5
sub.to_csv('submission.csv',index=False)
sub

,comment_id,score
0,114890,1.069868
1,732895,4.585859
2,1139051,0.654489
3,1434512,0.889620
4,2084821,3.021303
...,...,...
7532,504235362,4.384260
7533,504235566,0.022335
7534,504308177,1.250839
7535,504570375,6.605672
